In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm 
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
def process(df):
    ranked_sentences = []
    df['sentences'] = df['sentences'].apply(eval)
    for i, row in tqdm(df.iterrows(),total=len(df)):
        s1 = df.loc[i]['sentences']
        s2 = df.loc[i]['summary']
        e1 = model.encode(s1)
        e2 = model.encode(s2)
        cos_sim = util.cos_sim(e2, e1)
        s1 = np.asarray(s1)
        ranks = s1[np.argsort(-1*cos_sim[0].numpy())]
        ranked_sentences.append(ranks.tolist())
    
    df['ranked-sentences'] = ranked_sentences
    return df


In [ ]:
base_path = "../results/summary/"
write_path = "../results/ranked/"

os.makedirs(write_path,exist_ok=True)
files = os.listdir(base_path)

for file in  files:
    summary_df=pd.read_csv(f"{base_path}/{file}")
    final_df=process(summary_df)
    file_name = file.split(".")[0]
    final_df.to_csv(f"{write_path}/{file_name}_ranked.csv",index=False)